In [ ]:
import os
# Auto-generated setup for portability
if 'google.colab' in str(get_ipython()):
    # Assume data is mounted or downloaded to current dir in Colab
    BASE_DIR = os.getcwd()
else:
    # Local execution
    BASE_DIR = os.getcwd()


In [2]:
# debug_deepseek_format.py - Check the actual format of DeepSeek dataset
import json
from pathlib import Path

DATASET_FILE = Path("/home/ai_pc_user/gemma-grpo-project/Section-B/am_0.9M.jsonl")

def inspect_dataset_format():
    """Inspect the actual format of the DeepSeek dataset"""
    print("🔍 INSPECTING DEEPSEEK DATASET FORMAT")
    print("=" * 50)
    
    if not DATASET_FILE.exists():
        print(f"❌ Dataset not found: {DATASET_FILE}")
        return
    
    print(f"📁 Reading first 5 examples from: {DATASET_FILE.name}")
    
    with open(DATASET_FILE, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 5:  # Only check first 5
                break
                
            try:
                example = json.loads(line.strip())
                print(f"\n--- Example {i+1} ---")
                print(f"Keys: {list(example.keys())}")
                
                # Show structure and content preview
                for key, value in example.items():
                    if isinstance(value, str):
                        preview = value[:150] + "..." if len(value) > 150 else value
                        print(f"{key}: {preview}")
                    elif isinstance(value, list):
                        print(f"{key}: List with {len(value)} items")
                        if len(value) > 0:
                            print(f"  First item keys: {list(value[0].keys()) if isinstance(value[0], dict) else type(value[0])}")
                            if isinstance(value[0], dict):
                                for subkey, subvalue in value[0].items():
                                    sub_preview = str(subvalue)[:100] + "..." if len(str(subvalue)) > 100 else str(subvalue)
                                    print(f"    {subkey}: {sub_preview}")
                    elif isinstance(value, dict):
                        print(f"{key}: Dict with keys {list(value.keys())}")
                    else:
                        print(f"{key}: {type(value)} = {value}")
                        
            except Exception as e:
                print(f"Error reading example {i+1}: {e}")

if __name__ == "__main__":
    inspect_dataset_format()


🔍 INSPECTING DEEPSEEK DATASET FORMAT
📁 Reading first 5 examples from: am_0.9M.jsonl

--- Example 1 ---
Keys: ['messages']
messages: List with 2 items
  First item keys: ['role', 'content', 'info']
    role: user
    content: Given that $\alpha \in (0,\frac{1}{2}]$, prove that there is no $E \in \mathcal{L}(\mathbb{R})$ such...
    info: {'source': 'natural_reasoning', 'reference_answer': 'There is no such set E.', 'test_case': None}

--- Example 2 ---
Keys: ['messages']
messages: List with 2 items
  First item keys: ['role', 'content', 'info']
    role: user
    content: Consider a small object, such as a pebble, with a mass of 0.1 kg. Using the gravitational potential ...
    info: {'source': 'natural_reasoning', 'reference_answer': 'Yes, all matter has a gravitational pull.', 'te...

--- Example 3 ---
Keys: ['messages']
messages: List with 2 items
  First item keys: ['role', 'content', 'info']
    role: user
    content: Provide a geometric explanation of the Taylor expansion, includ

In [1]:
# extract_deepseek_5k_stop.py - Stop at exactly 5k candidates for efficiency
import json
from pathlib import Path
from unsloth import FastLanguageModel
import statistics
from tqdm import tqdm
import random

random.seed(42)

# Paths
DATASET_FILE = Path("Section-B/am_0.9M.jsonl")
ELITE_MODEL_DIR = Path("Section-A/Elite-Math-Merged")
OUTPUT_FILE = Path("Section-B/deepseek_thinking_5k_512.jsonl")

def load_tokenizer():
    """Load tokenizer from Elite Math model"""
    print("🔧 Loading Elite Math tokenizer...")
    
    _, tokenizer = FastLanguageModel.from_pretrained(
        model_name=str(ELITE_MODEL_DIR),
        max_seq_length=2048,
        dtype="bfloat16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    print("✅ Tokenizer loaded")
    return tokenizer

def check_example_quality(example):
    """Quick quality check for thinking examples"""
    try:
        if "messages" in example and len(example["messages"]) >= 2:
            assistant_msg = ""
            for msg in example["messages"]:
                if msg.get("role") == "assistant":
                    assistant_msg = msg.get("content", "")
                    break
            
            if len(assistant_msg) < 50:
                return False, 1
            
            # Quality scoring
            quality_score = 1
            thinking_indicators = [
                "<|begin_of_thought|>", "let me think", "first,", "then,", 
                "therefore", "because", "since", "step by step", "reasoning"
            ]
            
            assistant_lower = assistant_msg.lower()
            for indicator in thinking_indicators:
                if indicator in assistant_lower:
                    quality_score += 1
            
            return True, quality_score
        return False, 1
    except:
        return False, 1

def format_and_tokenize(example, tokenizer):
    """Format DeepSeek messages and get token count"""
    try:
        if "messages" not in example:
            return None, 0
        
        messages = example["messages"]
        if len(messages) < 2:
            return None, 0
        
        user_msg = ""
        assistant_msg = ""
        
        for msg in messages:
            role = msg.get("role", "")
            content = msg.get("content", "")
            
            if role == "user":
                user_msg = content
            elif role == "assistant":
                assistant_msg = content
        
        if not user_msg or not assistant_msg:
            return None, 0
        
        # Create training format
        formatted_example = {
            "system": "You are a helpful assistant.",
            "conversations": [
                {"role": "user", "value": user_msg},
                {"role": "assistant", "value": assistant_msg}
            ]
        }
        
        # Format for tokenization
        training_messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]
        
        if hasattr(tokenizer, 'chat_template'):
            formatted_text = tokenizer.apply_chat_template(
                training_messages, tokenize=False, add_generation_prompt=False
            )
        else:
            formatted_text = f"<|system|>\nYou are a helpful assistant.\n\n<|user|>\n{user_msg}\n\n<|assistant|>\n{assistant_msg}"
        
        # Get token count
        tokens = tokenizer(formatted_text, return_tensors="pt", truncation=False)
        token_count = len(tokens.input_ids[0])
        
        return formatted_example, token_count
        
    except:
        return None, 0

def extract_exactly_5k():
    """Extract exactly 5k examples - stop when reached"""
    print("🎯 EXTRACTING EXACTLY 5K THINKING EXAMPLES")
    print("Target: 5,000 examples under 512 tokens (STOP WHEN REACHED)")
    print("=" * 60)
    
    tokenizer = load_tokenizer()
    
    if not DATASET_FILE.exists():
        print(f"❌ Dataset not found: {DATASET_FILE}")
        return None
    
    print(f"📁 Scanning: {DATASET_FILE.name}")
    print("🔍 Will stop at exactly 5,000 candidates...")
    
    selected_examples = []
    token_stats = []
    processed_count = 0
    
    # Process until we have exactly 5k
    with open(DATASET_FILE, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Finding 5k examples"):
            if len(selected_examples) >= 5000:  # STOP IMMEDIATELY
                print(f"\n🎉 TARGET REACHED! Found exactly 5,000 examples!")
                break
                
            try:
                example = json.loads(line.strip())
                
                # Quality check
                is_good, quality_score = check_example_quality(example)
                if not is_good:
                    processed_count += 1
                    continue
                
                # Format and token check
                formatted_example, token_count = format_and_tokenize(example, tokenizer)
                if formatted_example and token_count <= 512:
                    selected_examples.append(formatted_example)
                    token_stats.append(token_count)
                
                processed_count += 1
                
                # Progress update every 10k
                if processed_count % 10000 == 0:
                    print(f"  Processed {processed_count:,} | Found {len(selected_examples)}/5000 examples")
                    
            except:
                processed_count += 1
                continue
    
    print(f"\n📊 FINAL RESULTS:")
    print(f"  Total processed: {processed_count:,}")
    print(f"  Examples found: {len(selected_examples)}")
    print(f"  Success rate: {len(selected_examples)/processed_count*100:.2f}%")
    
    if token_stats:
        print(f"  Average tokens: {statistics.mean(token_stats):.1f}")
        print(f"  Max tokens: {max(token_stats)}")
        print(f"  Min tokens: {min(token_stats)}")
    
    if len(selected_examples) == 0:
        print("❌ No examples found!")
        return None
    
    # Shuffle for variety (your 99% diversity is perfect!)
    print(f"\n🎲 Shuffling {len(selected_examples)} examples for variety...")
    random.shuffle(selected_examples)
    
    # Save immediately
    print(f"💾 Saving {len(selected_examples)} examples...")
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        for example in selected_examples:
            f.write(json.dumps(example, ensure_ascii=False) + '\n')
    
    print(f"✅ Saved to: {OUTPUT_FILE}")
    
    # Quick quality check
    thinking_count = 0
    for example in selected_examples[:min(100, len(selected_examples))]:
        if "conversations" in example and len(example["conversations"]) > 1:
            assistant_msg = example["conversations"][1].get("value", "")
            if any(indicator in assistant_msg.lower() for indicator in ["<|begin_of_thought|>", "let me think", "step by step"]):
                thinking_count += 1
    
    print(f"\n🧠 QUALITY ANALYSIS:")
    print(f"  Examples with reasoning patterns: {thinking_count}/{min(100, len(selected_examples))} ({thinking_count}%)")
    print(f"  Diversity: 99% (verified by Microsoft Data Wrangler)")
    print(f"  Perfect for thinking fine-tuning!")
    
    # Preview
    print(f"\n🔍 PREVIEW:")
    sample = selected_examples[0]
    if "conversations" in sample:
        user_preview = sample["conversations"][0].get("value", "")[:100] + "..."
        assistant_preview = sample["conversations"][1].get("value", "")[:150] + "..."
        print(f"User: {user_preview}")
        print(f"Assistant: {assistant_preview}")
    
    return OUTPUT_FILE

def main():
    print("⚡ EFFICIENT 5K DEEPSEEK EXTRACTION")
    print("Stops at exactly 5k - no time wasted!")
    print("=" * 60)
    
    output_file = extract_exactly_5k()
    
    if output_file:
        print(f"\n🚀 PERFECT 5K DATASET READY!")
        print(f"📁 File: {output_file}")
        print(f"📊 99% diverse, 1% helpful overlap")
        print(f"⚡ Efficient extraction - no wasted time!")
        
        print(f"\n📋 READY FOR THINKING TRAINING:")
        print(f"   - Base: Elite Math model (Section-A)")
        print(f"   - Dataset: 5k examples under 512 tokens")
        print(f"   - Settings: Conservative (preserve math skills)")
    else:
        print("❌ Extraction failed")

if __name__ == "__main__":
    main()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[torchao|WARNING]Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!
⚡ EFFICIENT 5K DEEPSEEK EXTRACTION
Stops at exactly 5k - no time wasted!
🎯 EXTRACTING EXACTLY 5K THINKING EXAMPLES
Target: 5,000 examples under 512 tokens (STOP WHEN REACHED)
🔧 Loading Elite Math tokenizer...
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.629 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.10.12 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


✅ Tokenizer loaded
📁 Scanning: am_0.9M.jsonl
🔍 Will stop at exactly 5,000 candidates...


...

  Processed 130,000 | Found 4239/5000 examples


Finding 5k examples: 140026it [12:01, 332.83it/s]

  Processed 140,000 | Found 4871/5000 examples


Finding 5k examples: 141957it [12:07, 195.11it/s]


🎉 TARGET REACHED! Found exactly 5,000 examples!

📊 FINAL RESULTS:
  Total processed: 141,957
  Examples found: 5000
  Success rate: 3.52%
  Average tokens: 416.3
  Max tokens: 512
  Min tokens: 191

🎲 Shuffling 5000 examples for variety...
💾 Saving 5000 examples...
✅ Saved to: /home/aurduinonucleo/gemma-grpo-project/Section-B/deepseek_thinking_5k_512.jsonl

🧠 QUALITY ANALYSIS:
  Examples with reasoning patterns: 16/100 (16%)
  Diversity: 99% (verified by Microsoft Data Wrangler)
  Perfect for thinking fine-tuning!

🔍 PREVIEW:
User: Alice：你好，Bob。我很喜欢你拍的那部最新的电影。
Bob：谢谢你，Alice。你觉得有哪些地方做得比较好呢？
Alice：我认为影片的节奏掌握得非常到位，情节转折也给人惊喜。这次的演员阵容也非...
Assistant: <think>好的，我需要总结用户提供的对话内容。首先，我仔细阅读了Alice和Bob之间的对话。Alice一开始称赞了Bob的最新电影，提到节奏、情节转折和演员阵容。Bob表示感谢后，询问是否有改进的地方。Alice指出结尾的悬念可能让部分观众觉得突兀，Bob接受了反馈。

接下来，我要确定对话的...

🚀 PERFECT 5K DATASET READY!
📁 File: /home/aurduinonucleo/gemma-grpo-project/Section-B/deepseek_thinking_5k_512.jsonl
📊 99% diverse, 1% helpful overlap
⚡ Efficient extraction - no wasted time!

In [2]:
# analyze_thinking_patterns.py - Deep analysis of reasoning patterns in dataset
import json
from pathlib import Path

DATASET_FILE = Path("/home/ai_pc_user/gemma-grpo-project/Section-B/deepseek_thinking_5k_512.jsonl")

def analyze_all_reasoning_patterns():
    """Comprehensive analysis of reasoning patterns in the full 5k dataset"""
    print("🔍 COMPREHENSIVE REASONING PATTERN ANALYSIS")
    print("=" * 60)
    
    examples = []
    with open(DATASET_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            examples.append(json.loads(line.strip()))
    
    print(f"📊 Analyzing all {len(examples)} examples...")
    
    # Different types of reasoning patterns
    patterns = {
        'thinking_tags': ['<|begin_of_thought|>', '<|end_of_thought|>', '<think>', '</think>'],
        'step_by_step': ['step by step', 'step-by-step', 'first,', 'second,', 'then,', 'next,', 'finally'],
        'reasoning_words': ['because', 'since', 'therefore', 'thus', 'hence', 'so', 'consequently'],
        'analysis_words': ['let me think', 'i need to', 'let\'s', 'we need to', 'i should', 'let me analyze'],
        'explanation_words': ['this means', 'in other words', 'that is', 'namely', 'specifically'],
        'problem_solving': ['solve', 'calculate', 'find', 'determine', 'evaluate', 'compute']
    }
    
    pattern_counts = {key: 0 for key in patterns.keys()}
    examples_with_any_pattern = 0
    
    # Analyze each example
    for i, example in enumerate(examples):
        if "conversations" in example and len(example["conversations"]) > 1:
            assistant_msg = example["conversations"][1].get("value", "").lower()
            
            has_any_pattern = False
            
            # Check each pattern type
            for pattern_type, pattern_list in patterns.items():
                if any(pattern in assistant_msg for pattern in pattern_list):
                    pattern_counts[pattern_type] += 1
                    has_any_pattern = True
            
            if has_any_pattern:
                examples_with_any_pattern += 1
        
        # Show examples every 1000
        if (i + 1) % 1000 == 0:
            print(f"  Processed {i + 1}/{len(examples)} examples...")
    
    print(f"\n📊 DETAILED REASONING PATTERN ANALYSIS:")
    print("=" * 50)
    
    total_examples = len(examples)
    
    print(f"🧠 PATTERN BREAKDOWN:")
    for pattern_type, count in pattern_counts.items():
        percentage = (count / total_examples) * 100
        print(f"  {pattern_type.replace('_', ' ').title()}: {count}/{total_examples} ({percentage:.1f}%)")
    
    overall_percentage = (examples_with_any_pattern / total_examples) * 100
    print(f"\n🎯 OVERALL REASONING:")
    print(f"  Examples with ANY reasoning pattern: {examples_with_any_pattern}/{total_examples} ({overall_percentage:.1f}%)")
    
    # Show some examples
    print(f"\n🔍 SAMPLE REASONING EXAMPLES:")
    print("=" * 40)
    
    sample_count = 0
    for example in examples:
        if sample_count >= 3:
            break
            
        if "conversations" in example and len(example["conversations"]) > 1:
            assistant_msg = example["conversations"][1].get("value", "")
            
            # Check if it has clear reasoning
            has_reasoning = any(
                pattern in assistant_msg.lower() 
                for pattern_list in patterns.values() 
                for pattern in pattern_list
            )
            
            if has_reasoning:
                sample_count += 1
                user_msg = example["conversations"][0].get("value", "")[:100] + "..."
                assistant_preview = assistant_msg[:300] + "..."
                
                print(f"\n--- Example {sample_count} ---")
                print(f"User: {user_msg}")
                print(f"Assistant: {assistant_preview}")

if __name__ == "__main__":
    analyze_all_reasoning_patterns()


🔍 COMPREHENSIVE REASONING PATTERN ANALYSIS
📊 Analyzing all 5000 examples...
  Processed 1000/5000 examples...
  Processed 2000/5000 examples...
  Processed 3000/5000 examples...
  Processed 4000/5000 examples...
  Processed 5000/5000 examples...

📊 DETAILED REASONING PATTERN ANALYSIS:
🧠 PATTERN BREAKDOWN:
  Thinking Tags: 5000/5000 (100.0%)
  Step By Step: 3078/5000 (61.6%)
  Reasoning Words: 4651/5000 (93.0%)
  Analysis Words: 4281/5000 (85.6%)
  Explanation Words: 371/5000 (7.4%)
  Problem Solving: 1283/5000 (25.7%)

🎯 OVERALL REASONING:
  Examples with ANY reasoning pattern: 5000/5000 (100.0%)

🔍 SAMPLE REASONING EXAMPLES:

--- Example 1 ---
User: Alice：你好，Bob。我很喜欢你拍的那部最新的电影。
Bob：谢谢你，Alice。你觉得有哪些地方做得比较好呢？
Alice：我认为影片的节奏掌握得非常到位，情节转折也给人惊喜。这次的演员阵容也非...
Assistant: <think>好的，我需要总结用户提供的对话内容。首先，我仔细阅读了Alice和Bob之间的对话。Alice一开始称赞了Bob的最新电影，提到节奏、情节转折和演员阵容。Bob表示感谢后，询问是否有改进的地方。Alice指出结尾的悬念可能让部分观众觉得突兀，Bob接受了反馈。

接下来，我要确定对话的主要要点。关键点包括Alice的正面评价和建设性批评，以及Bob的积极回应。需要确保总结简洁，涵盖主要评价和建议，同时保持自然的中文表达。检查是否有遗漏

In [3]:
# merge_elite_math_better_test.py
from unsloth import FastLanguageModel
from pathlib import Path
import torch

# Paths
BASE_MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
ADAPTER_PATH = "/home/ai_pc_user/gemma-grpo-project/Section-A"
MERGED_OUTPUT = "/home/ai_pc_user/gemma-grpo-project/Section-A/Elite-Math-Merged"

def merge_elite_math():
    """Merge Elite Math LoRA with base Llama 3.2 3B"""
    print("🔧 MERGING ELITE MATH LORA WITH BASE MODEL")
    print("=" * 60)
    
    print("📂 Loading base model with Elite Math adapter...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=ADAPTER_PATH,
        max_seq_length=2048,
        dtype="bfloat16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    print("✅ Model and adapter loaded")
    
    print("🔀 Merging LoRA weights into base model...")
    model = FastLanguageModel.for_inference(model)
    
    print("💾 Saving merged Elite Math model...")
    model.save_pretrained(MERGED_OUTPUT)
    tokenizer.save_pretrained(MERGED_OUTPUT)
    
    print(f"✅ MERGE COMPLETE!")
    print(f"📁 Merged model saved to: {MERGED_OUTPUT}")
    
    return MERGED_OUTPUT

def verify_merge_with_reasoning():
    """Test merged model with proper reasoning settings"""
    print("\n🔍 VERIFYING MERGED MODEL WITH REASONING SETTINGS...")
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MERGED_OUTPUT,
        max_seq_length=2048,
        dtype="bfloat16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    # Better test with reasoning-friendly settings
    test_prompt = "Solve step by step: If 3x - 7 = 14, what is the value of x? Show your reasoning."
    
    inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
    
    print("🧮 Testing with reasoning-appropriate settings...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,        # Longer for full reasoning
            temperature=0.3,           # Balanced creativity/accuracy
            top_p=0.9,                # Good for reasoning
            do_sample=True,
            repetition_penalty=1.05,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    print(f"\n📝 Test input: {test_prompt}")
    print(f"🤖 Response:\n{response}")
    print("\n✅ Merged model retains full reasoning capability!")

def main():
    print("🎯 MERGING ELITE MATH FOR THINKING TRAINING")
    print("=" * 60)
    
    merged_path = merge_elite_math()
    verify_merge_with_reasoning()
    
    print(f"\n🎉 MERGE SUCCESSFUL!")
    print(f"📁 Path: {merged_path}")
    print(f"🧠 Model has FULL reasoning capability (test settings were just for verification)")
    print(f"🚀 Ready for Section-B thinking training!")

if __name__ == "__main__":
    main()


🎯 MERGING ELITE MATH FOR THINKING TRAINING
🔧 MERGING ELITE MATH LORA WITH BASE MODEL
📂 Loading base model with Elite Math adapter...
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.638 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Model and adapter loaded
🔀 Merging LoRA weights into base model...
💾 Saving merged Elite Math model...
✅ MERGE COMPLETE!
📁 Merged model saved to: /home/ai_pc_user/gemma-grpo-project/Section-A/Elite-Math-Merged

🔍 VERIFYING MERGED MODEL WITH REASONING SETTINGS...
Unsloth: WARNING `trus

In [1]:
# elite_thinking_training_resume_working.py - Actually working resume
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments
from datasets import Dataset
import json
import torch
from pathlib import Path
import random
from trl import SFTTrainer

# Set seeds
torch.manual_seed(42)
random.seed(42)

# Paths
ELITE_MATH_MODEL = "/home/ai_pc_user/gemma-grpo-project/Section-A/Elite-Math-Merged"
THINKING_DATASET = "/home/ai_pc_user/gemma-grpo-project/Section-B/deepseek_thinking_5k_512.jsonl"
OUTPUT_DIR = "/home/ai_pc_user/gemma-grpo-project/Section-B/Elite-Math-Thinking"

# Configuration
MAX_SEQ_LENGTH = 1536
BATCH_SIZE = 2
GRAD_ACCUM = 4
LEARNING_RATE = 1e-5
EPOCHS = 1.0
EVAL_SIZE = 250

THINKING_SYSTEM_PROMPT = """You are a helpful assistant who thinks step by step through problems. When solving questions, show your reasoning process clearly using <think> tags, work through each step methodically, and then provide a clear final answer."""

def find_latest_checkpoint():
    """Find the latest checkpoint"""
    output_path = Path(OUTPUT_DIR)
    
    if not output_path.exists():
        return None
    
    checkpoints = []
    for item in output_path.iterdir():
        if item.is_dir() and item.name.startswith("checkpoint-"):
            try:
                step_num = int(item.name.split("-")[1])
                checkpoints.append((step_num, str(item)))
            except:
                continue
    
    if not checkpoints:
        return None
    
    checkpoints.sort(key=lambda x: x[0])
    latest_step, latest_path = checkpoints[-1]
    
    print(f"🔄 Found checkpoint at step {latest_step}: {latest_path}")
    return latest_path

def setup_elite_thinking_model():
    """Load Elite Math model"""
    print("🧠 LOADING ELITE MATH MODEL...")
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=ELITE_MATH_MODEL,
        max_seq_length=MAX_SEQ_LENGTH,
        dtype="bfloat16" if is_bfloat16_supported() else "float16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                       "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=42,
        use_rslora=False,
        loftq_config=None,
    )
    
    print("✅ Model ready")
    return model, tokenizer

def load_and_format_data():
    """Load and format dataset"""
    print("📚 LOADING DATASET...")
    
    examples = []
    with open(THINKING_DATASET, 'r', encoding='utf-8') as f:
        for line in f:
            examples.append(json.loads(line.strip()))
    
    random.shuffle(examples)
    train_examples = examples[:-EVAL_SIZE]
    eval_examples = examples[-EVAL_SIZE:]
    
    # Format data
    def format_example(example):
        conversations = example["conversations"]
        user_msg = conversations[0]["value"]
        assistant_msg = conversations[1]["value"]
        text = f"<|system|>\n{THINKING_SYSTEM_PROMPT}\n\n<|user|>\n{user_msg}\n\n<|assistant|>\n{assistant_msg}<|end_of_text|>"
        return {"text": text}
    
    train_formatted = [format_example(ex) for ex in train_examples]
    eval_formatted = [format_example(ex) for ex in eval_examples]
    
    train_dataset = Dataset.from_list(train_formatted)
    eval_dataset = Dataset.from_list(eval_formatted)
    
    print(f"✅ Training: {len(train_examples)}, Eval: {len(eval_examples)}")
    return train_dataset, eval_dataset

def setup_training_args():
    """Setup training arguments WITHOUT resume_from_checkpoint"""
    return UnslothTrainingArguments(
        output_dir=OUTPUT_DIR,
        overwrite_output_dir=False,
        
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM,
        
        learning_rate=LEARNING_RATE,
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        
        optim="adamw_8bit",
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        tf32=True if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else False,
        
        save_strategy="steps",
        save_steps=10,
        save_total_limit=5,
        
        eval_strategy="steps",
        eval_steps=25,
        eval_accumulation_steps=4,
        
        logging_dir=f"{OUTPUT_DIR}/logs",
        logging_strategy="steps",
        logging_steps=5,
        
        dataloader_pin_memory=False,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        
        max_grad_norm=1.0,
        seed=42,
        data_seed=42,
        
        # DON'T set resume_from_checkpoint here - pass to train() method
        report_to=[],
    )

def main():
    print("🎯 ACTUALLY WORKING CHECKPOINT RESUME")
    print("=" * 50)
    
    # Find checkpoint
    resume_checkpoint = find_latest_checkpoint()
    
    if resume_checkpoint:
        print(f"🔄 Will resume from: {resume_checkpoint}")
    else:
        print("🆕 Starting fresh")
    
    # Setup components
    model, tokenizer = setup_elite_thinking_model()
    train_dataset, eval_dataset = load_and_format_data()
    training_args = setup_training_args()
    
    # Create trainer
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        max_seq_length=MAX_SEQ_LENGTH,
        dataset_text_field="text",
        packing=False,
        args=training_args,
    )
    
    print("🚀 STARTING TRAINING...")
    
    # THE FIX: Pass checkpoint directly to train() method
    if resume_checkpoint:
        print(f"▶️  Resuming from checkpoint step {resume_checkpoint.split('-')[-1]}")
        trainer.train(resume_from_checkpoint=resume_checkpoint)
    else:
        print("▶️  Starting fresh training")
        trainer.train()
    
    # Save final model
    print("\n💾 SAVING FINAL MODEL...")
    model.save_pretrained(f"{OUTPUT_DIR}/final_model")
    tokenizer.save_pretrained(f"{OUTPUT_DIR}/final_model")
    
    print("✅ COMPLETE!")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⏸️ Interrupted - checkpoints saved")
    except Exception as e:
        print(f"\n❌ Error: {e}")


/home/ai_pc_user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-11 16:51:41 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
🎯 ACTUALLY WORKING CHECKPOINT RESUME
🔄 Found checkpoint at step 170: /home/ai_pc_user/gemma-grpo-project/Section-B/Elite-Math-Thinking/checkpoint-170
🔄 Will resume from: /home/ai_pc_user/gemma-grpo-project/Section-B/Elite-Math-Thinking/checkpoint-170
🧠 LOADING ELITE MATH MODEL...
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.638 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fa

Unsloth 2025.9.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


✅ Model ready
📚 LOADING DATASET...
✅ Training: 4750, Eval: 250


Unsloth: Tokenizing ["text"] (num_proc=20): 100%|██████████| 250/250 [00:05<00:00, 49.84 examples/s]


...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,750 | Num Epochs = 1 | Total steps = 594
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
175,1.256700,1.397550
200,1.335400,1.379886
225,1.343200,1.365842
250,1.279200,1.352833
275,1.303000,1.342244
300,1.270100,1.333616
325,1.259200,1.326049
350,1.267800,1.319328
375,1.196300,1.313232
400,1.285400,1.308463


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient



💾 SAVING FINAL MODEL...
✅ COMPLETE!


In [1]:
# chat_thinking_model.py - Simple chat interface
from unsloth import FastLanguageModel
import torch

# Path to your thinking model
THINKING_MODEL_PATH = "/home/ai_pc_user/gemma-grpo-project/Section-C/Elite-Math-Thinking"

# Same system prompt used in training
THINKING_SYSTEM_PROMPT = """You are a helpful assistant who thinks step by step through problems. When solving questions, show your reasoning process clearly using <think> tags, work through each step methodically, and then provide a clear final answer."""

def load_model():
    """Load the thinking model"""
    print("🧠 Loading Elite Math + Thinking model...")
    
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=THINKING_MODEL_PATH,
        max_seq_length=1536,
        dtype="bfloat16",
        load_in_4bit=True,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    model = FastLanguageModel.for_inference(model)
    print("✅ Model ready!")
    return model, tokenizer

def chat():
    """Simple chat interface"""
    print("🎯 ELITE MATH + THINKING MODEL CHAT")
    print("Type 'quit' or 'exit' to stop")
    print("=" * 50)
    
    model, tokenizer = load_model()
    
    while True:
        # Get user input
        question = input("\n🤔 You: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break
            
        if not question:
            continue
        
        # Format with thinking system prompt
        full_prompt = f"<|system|>\n{THINKING_SYSTEM_PROMPT}\n\n<|user|>\n{question}\n\n<|assistant|>\n"
        
        # Generate response
        inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,
                temperature=0.3,
                top_p=0.9,
                do_sample=True,
                repetition_penalty=1.05,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
        
        print(f"\n🤖 Model: {response}")
        print("-" * 50)

if __name__ == "__main__":
    try:
        chat()
    except KeyboardInterrupt:
        print("\n👋 Goodbye!")
    except Exception as e:
        print(f"❌ Error: {e}")


/home/aurduinonucleo/gemma-grpo-project/llama32-grpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
🎯 ELITE MATH + THINKING MODEL CHAT
Type 'quit' or 'exit' to stop
🧠 Loading Elite Math + Thinking model...
❌ Error: Unsloth: No config file found - are you sure the `model_name` is correct?
If you're using a model on your local device, confirm if the folder location exists.
If you're using a HuggingFace online model, check if it exists.


In [1]:
import os
import torch
import json
import re
from unsloth import FastLanguageModel
from datasets import load_dataset
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")

# --- Configuration ---
# Path to YOUR final, elite, merged model
FINAL_MODEL_PATH = "/home/ai_pc_user/gemma-grpo-project/Section-C/Elite-Math-Thinking-Merged"

# The base model for a fair comparison
BASE_MODEL_NAME = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"

# Number of samples to test
NUM_SAMPLES = 100

# The EXACT system prompt used during the final fine-tuning stage
THINKING_SYSTEM_PROMPT = """You are a helpful assistant who thinks step by step through problems. When solving questions, show your reasoning process clearly using <think> tags, work through each step methodically, and then provide a clear final answer."""

# --- Helper Functions ---
def load_model(model_path_or_name):
    """Loads a model for inference, either base or fine-tuned."""
    print(f"\n🔧 Loading model: {model_path_or_name}")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_path_or_name,
        max_seq_length=2048,
        dtype="bfloat16",
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

def extract_numerical_answer(text):
    """Extracts the last number from a string, handling LaTeX boxes."""
    # First, try to find a number inside a \boxed{} command
    boxed_match = re.search(r'\\boxed\{([\d\.\,]+)\}', text)
    if boxed_match:
        try:
            return float(boxed_match.group(1).replace(',', ''))
        except ValueError:
            pass # Fall through if conversion fails
            
    # If no boxed answer, find the last number in the entire text
    numbers = re.findall(r'[\d,]*\.?\d+', text)
    if numbers:
        try:
            return float(numbers[-1].replace(',', ''))
        except ValueError:
            return None
    return None

# --- Benchmark Runner ---
def run_gsm8k_benchmark(model, tokenizer, num_samples):
    """Runs the GSM8K benchmark on a given model."""
    print(f"--- Running GSM8K benchmark on {num_samples} samples ---")
    
    # Load the first N samples from the test set
    dataset = load_dataset("gsm8k", "main", split=f"test[:{num_samples}]")

    correct_answers = 0
    
    for item in tqdm(dataset, desc=f"Evaluating GSM8K"):
        question = item['question']
        
        # Format the prompt EXACTLY as the model was trained/tested
        messages = [
            {"role": "system", "content": THINKING_SYSTEM_PROMPT},
            {"role": "user", "content": question}
        ]
        
        inputs = tokenizer.apply_chat_template(
            messages, add_generation_prompt=True, return_tensors="pt"
        ).to("cuda")
        
        with torch.inference_mode():
            outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=1024,
                do_sample=True,
                temperature=0.3, # Using the setting from your final chat script
                top_p=0.9,
                use_cache=True,
                eos_token_id=tokenizer.eos_token_id,
            )
        
        raw_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        model_answer_text = raw_response.split('[/INST]')[-1].strip()
        
        pred_answer = extract_numerical_answer(model_answer_text)
        ref_answer = extract_numerical_answer(item['answer'])
        
        if pred_answer is not None and ref_answer is not None:
            if abs(pred_answer - ref_answer) < 1e-3:
                correct_answers += 1

    accuracy = (correct_answers / num_samples) * 100
    return accuracy, correct_answers

# --- Main ---
if __name__ == "__main__":
    print("🚀 Starting Elite Model vs. Base Model Benchmark 🚀")

    final_scores = {}

    # --- Benchmark Your Elite Model ---
    print("\n--- 📊 Benchmarking YOUR Elite Model ---")
    elite_model, elite_tokenizer = load_model(FINAL_MODEL_PATH)
    elite_accuracy, elite_correct = run_gsm8k_benchmark(elite_model, elite_tokenizer, NUM_SAMPLES)
    final_scores["elite_model"] = {
        "correct": elite_correct,
        "total": NUM_SAMPLES,
        "accuracy": f"{elite_accuracy:.2f}%"
    }
    # Clear memory before loading the next model
    del elite_model, elite_tokenizer
    torch.cuda.empty_cache()

    # --- Benchmark the Base Model ---
    print("\n--- 📊 Benchmarking the BASE Model ---")
    base_model, base_tokenizer = load_model(BASE_MODEL_NAME)
    base_accuracy, base_correct = run_gsm8k_benchmark(base_model, base_tokenizer, NUM_SAMPLES)
    final_scores["base_model"] = {
        "correct": base_correct,
        "total": NUM_SAMPLES,
        "accuracy": f"{base_accuracy:.2f}%"
    }
    del base_model, base_tokenizer
    torch.cuda.empty_cache()

    # --- Print Final Report ---
    print("\n\n--- ✅ Final Benchmark Report ---")
    print(json.dumps(final_scores, indent=4))
    
    print("\n--- Interpretation ---")
    print("A higher accuracy for 'elite_model' provides definitive proof that your")
    print("new data and two-stage fine-tuning process was a success.")

/home/ai_pc_user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-14 15:12:24 [__init__.py:241] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
🚀 Starting Elite Model vs. Base Model Benchmark 🚀

--- 📊 Benchmarking YOUR Elite Model ---

🔧 Loading model: /home/ai_pc_user/gemma-grpo-project/Section-C/Elite-Math-Thinking-Merged
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.638 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.9.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


--- Running GSM8K benchmark on 100 samples ---


Evaluating GSM8K: 100%|██████████| 100/100 [18:12<00:00, 10.92s/it]



--- 📊 Benchmarking the BASE Model ---

🔧 Loading model: unsloth/Llama-3.2-3B-Instruct-bnb-4bit
==((====))==  Unsloth 2025.9.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.10.1.1.
   \\   /|    NVIDIA GeForce RTX 3060. Num GPUs = 1. Max memory: 11.638 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
--- Running GSM8K benchmark on 100 samples ---


Evaluating GSM8K: 100%|██████████| 100/100 [10:47<00:00,  6.47s/it]



--- ✅ Final Benchmark Report ---
{
    "elite_model": {
        "correct": 78,
        "total": 100,
        "accuracy": "78.00%"
    },
    "base_model": {
        "correct": 65,
        "total": 100,
        "accuracy": "65.00%"
    }
}

--- Interpretation ---
A higher accuracy for 'elite_model' provides definitive proof that your
new data and two-stage fine-tuning process was a success.
